In [1]:
# combine charts.csv and hot100.csv
import pandas as pd
from termcolor import colored

path_hot100 = '../data/ranking_5/hot100.csv'
path_charts = '../data/ranking_5/charts.csv'

# read&load data
try:
    df_hot = pd.read_csv(path_hot100, dtype={4: str})
    df_charts = pd.read_csv(path_charts)
    print(colored(f"Success！\nHot100 shape: {df_hot.shape}\nCharts shape: {df_charts.shape}", 'green'))
except FileNotFoundError:
    print(colored("Wrong!", 'red'))

Success！
Hot100 shape: (350787, 8)
Charts shape: (330087, 7)


In [2]:
# unify column name
df_hot = df_hot.drop(columns=['Image URL'])
df_hot.columns = ['date', 'song', 'artist', 'rank', 'last_week', 'peak_rank', 'weeks_on_board']
df_charts.columns = ['date', 'rank', 'song', 'artist', 'last_week', 'peak_rank', 'weeks_on_board']

In [3]:
# modify data type
num_col = ['rank', 'last_week', 'weeks_on_board', 'peak_rank']
for col in num_col:
    df_hot[col] = pd.to_numeric(df_hot[col], errors='coerce')
    df_charts[col] = pd.to_numeric(df_charts[col], errors='coerce')

df_hot['date'] = pd.to_datetime(df_hot['date'])
df_charts['date'] = pd.to_datetime(df_charts['date'])

df_hot[['song', 'artist']] = df_hot[['song', 'artist']].astype(str)
df_charts[['song', 'artist']] = df_charts[['song', 'artist']].astype(str)

print(df_hot.dtypes)
print(df_charts.dtypes)

date              datetime64[ns]
song                      object
artist                    object
rank                       int64
last_week                float64
peak_rank                  int64
weeks_on_board           float64
dtype: object
date              datetime64[ns]
rank                       int64
song                      object
artist                    object
last_week                float64
peak_rank                  int64
weeks_on_board             int64
dtype: object


In [4]:
# overall comparison
df_hot.info()
df_hot.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350787 entries, 0 to 350786
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            350787 non-null  datetime64[ns]
 1   song            350787 non-null  object        
 2   artist          350787 non-null  object        
 3   rank            350787 non-null  int64         
 4   last_week       350785 non-null  float64       
 5   peak_rank       350787 non-null  int64         
 6   weeks_on_board  315335 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 18.7+ MB


,date,rank,last_week,peak_rank,weeks_on_board
count,350787,350787.000000,350785.000000,350787.000000,315335.000000
mean,1992-03-14 16:55:33.469028096,50.500882,50.486894,32.836074,10.293596
min,1958-08-06 00:00:00,1.000000,1.000000,1.000000,2.000000
25%,1975-05-21 00:00:00,26.000000,25.000000,6.000000,5.000000
50%,1992-03-18 00:00:00,51.000000,51.000000,27.000000,8.000000
75%,2009-01-07 00:00:00,76.000000,76.000000,55.000000,14.000000
max,2025-10-22 00:00:00,100.000000,100.000000,100.000000,112.000000
std,NaN,28.866097,28.919824,27.718423,8.066920


In [5]:
df_charts.info()
df_charts.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  datetime64[ns]
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last_week       297775 non-null  float64       
 5   peak_rank       330087 non-null  int64         
 6   weeks_on_board  330087 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 17.6+ MB


,date,rank,last_week,peak_rank,weeks_on_board
count,330087,330087.000000,297775.000000,330087.000000,330087.000000
mean,1990-03-23 22:05:10.761708160,50.500929,47.591631,40.970629,9.161785
min,1958-08-04 00:00:00,1.000000,1.000000,1.000000,1.000000
25%,1974-06-01 00:00:00,26.000000,23.000000,13.000000,4.000000
50%,1990-03-24 00:00:00,51.000000,47.000000,38.000000,7.000000
75%,2006-01-14 00:00:00,76.000000,72.000000,65.000000,13.000000
max,2021-11-06 00:00:00,100.000000,100.000000,100.000000,90.000000
std,NaN,28.866094,28.054360,29.347481,7.618264


In [6]:
# check the null value
print(df_hot.isnull().sum())
print(df_charts.isnull().sum())

date                  0
song                  0
artist                0
rank                  0
last_week             2
peak_rank             0
weeks_on_board    35452
dtype: int64
date                  0
rank                  0
song                  0
artist                0
last_week         32312
peak_rank             0
weeks_on_board        0
dtype: int64


In [8]:
# combine two dataframes

# Ensure date column is in the correct datetime format
df_hot['date'] = pd.to_datetime(df_hot['date'])
df_charts['date'] = pd.to_datetime(df_charts['date'])

# Transform hot100: Group by date and sort by rank within each date
df_hot = df_hot.sort_values(by=['date', 'rank'], ascending=[True, True]).reset_index(drop=True)

# Align all dates to the nearest Monday (Standardizing chart cycles)
for df in [df_hot, df_charts]:
    # .dt.weekday returns 0-6 (0=Monday); subtracting this value aligns the date to the nearest Monday
    df['date'] = df['date'] - pd.to_timedelta(df['date'].dt.weekday, unit='D')


def create_clean_key(df):
    # Convert to lowercase -> keep alphanumeric characters only -> remove extra whitespaces.
    clean_song = df['song'].astype(str).str.lower().str.replace(r'[^a-z0-9]', '', regex=True)
    clean_artist = df['artist'].astype(str).str.lower().str.replace(r'[^a-z0-9]', '', regex=True)
    return clean_song + "_" + clean_artist


# Generate unique matching keys
df_hot['match_key'] = create_clean_key(df_hot)
df_charts['match_key'] = create_clean_key(df_charts)

# Establish a MultiIndex
df_hot.set_index(['date', 'match_key'], inplace=True)
df_charts.set_index(['date', 'match_key'], inplace=True)

# Complementary merge: Use df_hot as primary and fill missing values from df_charts
df_final = df_hot.combine_first(df_charts)

# Reset index and perform cleanup
df_final = df_final.reset_index()
df_final.drop(columns=['match_key'], inplace=True)

# Final sort: Ensure records are organized by date and rank.
df_final = df_final.sort_values(['date', 'rank']).reset_index(drop=True)

# Print summary of the merge results
print(f"Original Hot100 row count: {len(df_hot)}")
print(f"Final merged row count: {len(df_final)}")
print("-" * 30)
print("Post-merge missing value statistics:")
print(df_final.isnull().sum())

原始 Hot100 行数: 350787
合并后最终行数: 353900
------------------------------
合并后缺失值统计:
date                 0
artist               0
last_week          384
peak_rank            0
rank                 0
song                 0
weeks_on_board    5592
dtype: int64
